# Settings

In [ ]:
!pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 35.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.7 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=313f525ee4d21a73256399ece3eec53b25c3c06b7a815eb758f35ae51487d790
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
!pip install -q transformers datasets folium==0.2.1 apex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.0/70.0 kB 8.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.8/75.8 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 709.8/709.8 kB 16.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.3/247.3 kB 13.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.5/136.5 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━

In [ ]:
import os
import time
import json
import shutil
import argparse
import numpy as np
from tqdm import tqdm
import random
import pandas as pd
from numpy import dot
from numpy.linalg import norm
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader

from transformers import AutoModel, AutoTokenizer, AutoConfig, BertPreTrainedModel, BertModel, BertConfig, BertTokenizer, BertTokenizerFast
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

from dataset import SelectionDataset
from transform import SelectionSequentialTransform, SelectionJoinTransform, SelectionConcatTransform
from encoder import PolyEncoder, BiEncoder, CrossEncoder

from sklearn.metrics import label_ranking_average_precision_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import logging

In [ ]:
args = {
        "bert_model": 'klue/bert-base',
        "eval": False,
        "model_type": 'bert',
        "output_dir": '/content/drive/MyDrive/',
        "train_dir": '/content/drive/MyDrive/',

        "use_pretrain": True,
        "architecture": 'poly',

        "max_contexts_length": 128,
        "max_response_length": 32,
        "train_batch_size": 32,
        "eval_batch_size": 32,
        "print_freq": 100,

        "poly_m": 16,

        "learning_rate": 5e-5,
        "weight_decay": 0.01,
        "warmup_steps": 100,
        "adam_epsilon": 1e-8,
        "max_grad_norm": 1.0,

        "num_train_epochs": 10.0,
        'seed': 12345,
        'gradient_accumulation_steps': 1,
        "fp16": False,
        "fp16_opt_level": "O1",
        'gpu': 0
        }

In [ ]:
from easydict import EasyDict as edict
args = edict(args)

In [ ]:
# 로깅 레벨을 ERROR로 설정
logging.basicConfig(level=logging.ERROR)

def set_seed(args):
    random.seed(args.seed)  # 랜덤 시드 설정
    np.random.seed(args.seed)  # NumPy 랜덤 시드 설정
    torch.manual_seed(args.seed)  # PyTorch 랜덤 시드 설정

os.environ["CUDA_VISIBLE_DEVICES"] = "%d" % args.gpu  # GPU 설정
set_seed(args)  # 시드 설정 함수 호출


# 데이터셋 및 모델 준비

In [ ]:
## 데이터셋 및 BERT 모델 초기화
model_name = args.bert_model
tokenizer = BertTokenizer.from_pretrained(model_name)  # BERT 토크나이저 초기화

# 데이터 변환 함수 초기화
context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=args.max_contexts_length)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=args.max_response_length)
concat_transform = SelectionConcatTransform(tokenizer=tokenizer, max_len=args.max_response_length+args.max_contexts_length)

print('=' * 80)
print('Train dir:', args.train_dir)
print('Output dir:', args.output_dir)
print('=' * 80)

if not args.eval:
    # 훈련 데이터셋 초기화
    train_dataset = SelectionDataset(os.path.join(args.train_dir, 'train.pickle'),
                                                                    context_transform, response_transform, concat_transform, sample_cnt=None, mode=args.architecture)
    val_dataset = SelectionDataset(os.path.join(args.train_dir, 'val.pickle'),
                                                                context_transform, response_transform, concat_transform, sample_cnt=None, mode=args.architecture)
    train_dataloader = DataLoader(train_dataset, batch_size=args.train_batch_size, collate_fn=train_dataset.batchify_join_str, shuffle=True, num_workers=0)
    t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
else: # 테스트
    val_dataset = SelectionDataset(os.path.join(args.train_dir, 'test.pickle'),
                                                                context_transform, response_transform, concat_transform, sample_cnt=None, mode=args.architecture)

val_dataloader = DataLoader(val_dataset, batch_size=args.eval_batch_size, collate_fn=val_dataset.batchify_join_str, shuffle=False, num_workers=0)


In [ ]:
# 모델 train시 필요한 파라미터 지정.

epoch_start = 1
global_step = 0
best_eval_loss = float('inf')
best_test_loss = float('inf')

if not os.path.exists(args.output_dir):
    os.makedirs(args.output_dir)

log_wf = open(os.path.join(args.output_dir, 'log.txt'), 'a', encoding='utf-8')
print(args, file=log_wf)

state_save_path = os.path.join(args.output_dir, '{}_{}_pytorch_model.bin'.format(args.architecture, args.poly_m))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
########################################
## BERT encoder build하기
########################################

bert_config = BertConfig.from_pretrained(model_name)

bert = BertModel.from_pretrained(model_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=args.poly_m)

model.resize_token_embeddings(len(tokenizer))
model.to(device)

# 학습 준비

### 모델 평가 함수 정의

In [ ]:
def eval_running_model(dataloader, test=False):
    """
    모델을 평가하고 평가 지표를 반환합니다.

    Args:
        dataloader (DataLoader): 평가 데이터를 로드하는 DataLoader 객체.
        test (bool, optional): 테스트 모드 여부. 기본값은 False.

    Returns:
        dict: 평가 결과를 담은 딕셔너리. 훈련 중이면 'train_loss'도 포함됩니다.
            - 'train_loss' (float): 훈련 손실 (훈련 중일 때만 포함).
            - 'eval_loss' (float): 평가 손실.
            - 'R1' (float): R@1 지표.
            - 'R2' (float): R@2 지표.
            - 'R5' (float): R@5 지표.
            - 'R10' (float): R@10 지표.
            - 'MRR' (float): 평균 상대 순위(MRR) 지표.
            - 'epoch' (int): 현재 에포크 (훈련 중일 때만 포함).
            - 'global_step' (int): 현재 전체 스텝 수 (훈련 중일 때만 포함).
    """
    model.eval()
    eval_loss, eval_hit_times = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    r10 = r2 = r1 = r5 = 0
    mrr = []
    for step, batch in enumerate(dataloader):
        batch = tuple(t.to(device) for t in batch)

        if args.architecture == 'cross':
            text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch = batch
            with torch.no_grad():
                logits = model(text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch)
                loss = F.cross_entropy(logits, torch.argmax(labels_batch, 1))

        else:
            context_token_ids_list_batch, context_input_masks_list_batch, \
            response_token_ids_list_batch, response_input_masks_list_batch, labels_batch = batch

            with torch.no_grad():
                logits = model(context_token_ids_list_batch, context_input_masks_list_batch,
                                              response_token_ids_list_batch, response_input_masks_list_batch)
                loss = F.cross_entropy(logits, torch.argmax(labels_batch, 1))

        r2_indices = torch.topk(logits, 2)[1] # R 2 @ 100
        r5_indices = torch.topk(logits, 5)[1] # R 5 @ 100
        r10_indices = torch.topk(logits, 10)[1] # R 10 @ 100

        r1 += (logits.argmax(-1) == 0).sum().item()
        r2 += ((r2_indices==0).sum(-1)).sum().item()
        r5 += ((r5_indices==0).sum(-1)).sum().item()
        r10 += ((r10_indices==0).sum(-1)).sum().item()

        # mrr
        logits = logits.data.cpu().numpy()
        for logit in logits:
            y_true = np.zeros(len(logit))
            y_true[0] = 1
            mrr.append(label_ranking_average_precision_score([y_true], [logit]))

        eval_loss += loss.item()
        nb_eval_examples += labels_batch.size(0)
        nb_eval_steps += 1

    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = r1 / nb_eval_examples

    if not test:
        result = {
            'train_loss': tr_loss / nb_tr_steps,
            'eval_loss': eval_loss,
            'R1': r1 / nb_eval_examples,
            'R2': r2 / nb_eval_examples,
            'R5': r5 / nb_eval_examples,
            'R10': r10 / nb_eval_examples,
            'MRR': np.mean(mrr),
            'epoch': epoch,
            'global_step': global_step,
        }

    else:
        result = {
            'eval_loss': eval_loss,
            'R1': r1 / nb_eval_examples,
            'R2': r2 / nb_eval_examples,
            'R5': r5 / nb_eval_examples,
            'R10': r10 / nb_eval_examples,
            'MRR': np.mean(mrr),
        }

    return result

### 학습 및 평가시 파라미터 정의

In [ ]:
if args.eval:
    print('Loading parameters from', state_save_path)
    model.load_state_dict(torch.load(state_save_path))  # 저장된 모델 파라미터 불러오기
    test_result = eval_running_model(val_dataloader, test=True)  # 평가 모드에서 모델 평가 실행
    print(test_result)  # 평가 결과 출력
    exit()  # 프로그램 종료

no_decay = ["bias", "LayerNorm.weight"]

# 옵티마이저 그룹화된 매개변수 설정
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]

# AdamW 옵티마이저 초기화
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)

# 스케줄러 초기화
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total
)

# 좀 더 빠른 실행시간을 원한다면? fp16 -> True
if args.fp16:
    try:
        from apex import amp
    except ImportError:
        raise ImportError("Please install apex from https://www.github.com/nvidia/apex to use fp16 training.")
    model, optimizer = amp.initialize(model, optimizer, opt_level=args.fp16_opt_level)


In [ ]:
# 출력 빈도와 평가 빈도 설정
print_freq = args.print_freq // args.gradient_accumulation_steps  # 출력 빈도 설정
eval_freq = min(len(train_dataloader) // 2, 1000)  # 평가 빈도 설정 (훈련 데이터의 절반 또는 최대 1000번)
eval_freq = eval_freq // args.gradient_accumulation_steps  # 평가 빈도를 경사 누적 단계로 나눔

# 설정된 빈도 출력
print('Print freq:', print_freq, "Eval freq:", eval_freq)


# Train

In [ ]:
for epoch in range(epoch_start, int(args.num_train_epochs) + 1):
    tr_loss = 0  # 훈련 손실 초기화
    nb_tr_steps = 0  # 훈련 스텝 수 초기화
    with tqdm(total=len(train_dataloader) // args.gradient_accumulation_steps) as bar:
        for step, batch in enumerate(train_dataloader):
            model.train()  # 모델을 훈련 모드로 설정
            optimizer.zero_grad()  # 그래디언트 초기화
            batch = tuple(t.to(device) for t in batch)
            if args.architecture == 'cross':
                text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch = batch
                loss = model(text_token_ids_list_batch, text_input_masks_list_batch, text_segment_ids_list_batch, labels_batch)
            else:
                context_token_ids_list_batch, context_input_masks_list_batch, \
                response_token_ids_list_batch, response_input_masks_list_batch, labels_batch = batch
                loss = model(context_token_ids_list_batch, context_input_masks_list_batch,
                                        response_token_ids_list_batch, response_input_masks_list_batch,
                                        labels_batch)

            loss = loss / args.gradient_accumulation_steps  # 그래디언트 누적을 고려한 손실

            if args.fp16:
                with amp.scale_loss(loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()

            tr_loss += loss.item()  # 훈련 손실 누적

            if (step + 1) % args.gradient_accumulation_steps == 0:
                if args.fp16:
                    torch.nn.utils.clip_grad_norm_(amp.master_params(optimizer), args.max_grad_norm)
                else:
                    torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
                nb_tr_steps += 1
                optimizer.step()
                scheduler.step()
                model.zero_grad()
                global_step += 1

                if nb_tr_steps and nb_tr_steps % print_freq == 0:
                    bar.update(min(print_freq, nb_tr_steps))
                    time.sleep(0.02)
                    print(global_step, tr_loss / nb_tr_steps)  # 훈련 스텝 및 현재 평균 훈련 손실 출력
                    log_wf.write('%d\t%f\n' % (global_step, tr_loss / nb_tr_steps))

                if global_step and global_step % eval_freq == 0:
                    val_result = eval_running_model(val_dataloader)
                    print('Global Step %d VAL res:\n' % global_step, val_result)  # 검증 결과 출력
                    log_wf.write('Global Step %d VAL res:\n' % global_step)
                    log_wf.write(str(val_result) + '\n')

                    if val_result['eval_loss'] < best_eval_loss:
                        best_eval_loss = val_result['eval_loss']
                        val_result['best_eval_loss'] = best_eval_loss
                        # 모델 저장
                        print('[Saving at]', state_save_path)
                        log_wf.write('[Saving at] %s\n' % state_save_path)
                        torch.save(model.state_dict(), state_save_path)

            log_wf.flush()


    # EVAL
    # 각 에포크 후에 평가 스텝 추가
    val_result = eval_running_model(val_dataloader)
    print('Epoch %d, Global Step %d VAL res:\n' % (epoch, global_step), val_result)  # 에포크별 검증 결과 출력
    log_wf.write('Global Step %d VAL res:\n' % global_step)
    log_wf.write(str(val_result) + '\n')

    if val_result['eval_loss'] < best_eval_loss:
        best_eval_loss = val_result['eval_loss']
        val_result['best_eval_loss'] = best_eval_loss
        # 모델 저장
        print('[Saving at]', state_save_path)
        log_wf.write('[Saving at] %s\n' % state_save_path)
        torch.save(model.state_dict(), state_save_path)
    print(global_step, tr_loss / nb_tr_steps)
    log_wf.write('%d\t%f\n' % (global_step, tr_loss / nb_tr_steps))


Train dir: /content/drive/MyDrive/
Output dir: /content/drive/MyDrive/


You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32001. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Print freq: 100 Eval freq: 136


  0%|          | 0/273 [00:00<?, ?it/s]/content/Poly-Encoder/dataset.py:107: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  labels_batch = torch.tensor(labels_batch, dtype=torch.long)
 37%|███▋      | 100/273 [00:47<01:21,  2.11it/s]

100 16.520170426368715
Global Step 136 VAL res:
 {'train_loss': 13.035817979013219, 'eval_loss': 1.7773750305175782, 'R1': 0.3605504587155963, 'R2': 0.5385321100917431, 'R5': 0.8623853211009175, 'R10': 1.0, 'MRR': 0.5572801077617592, 'epoch': 1, 'global_step': 136}
[Saving at] /content/drive/MyDrive/poly_16_pytorch_model.bin


...

2557 0.5887033489346504
Global Step 2584 VAL res:
 {'train_loss': 0.5986467437950644, 'eval_loss': 0.9157308114373206, 'R1': 0.826605504587156, 'R2': 0.9174311926605505, 'R5': 0.9880733944954129, 'R10': 1.0, 'MRR': 0.8937265181301878, 'epoch': 10, 'global_step': 2584}


 73%|███████▎  | 200/273 [01:40<00:37,  1.95it/s]

2657 0.6133759367465973
Global Step 2720 VAL res:
 {'train_loss': 0.6112691159767343, 'eval_loss': 0.9299156976556333, 'R1': 0.8293577981651377, 'R2': 0.9192660550458716, 'R5': 0.9899082568807339, 'R10': 1.0, 'MRR': 0.8954517984563857, 'epoch': 10, 'global_step': 2720}


 73%|███████▎  | 200/273 [02:24<00:52,  1.39it/s]


Epoch 10, Global Step 2730 VAL res:
 {'train_loss': 0.6136849342739625, 'eval_loss': 0.9301141640927848, 'R1': 0.8293577981651377, 'R2': 0.918348623853211, 'R5': 0.9899082568807339, 'R10': 1.0, 'MRR': 0.8952988932576088, 'epoch': 10, 'global_step': 2730}
2730 0.6136849342739625


# Test

## candidates data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/poly-encoder답변모음.csv')
df = df.drop(columns=['Unnamed: 0'])
data = df.iloc[:-4758]
data = data[data['answer'].notna() & (data['answer'] != ' ')]
data

,answer
0,"공모주는 상장 후 보유하셔도 되고, 매도하셔도 됩니다."
1,공모가는 개인과 기관 둘 다 동일합니다.
2,주식 계좌의 예수금은 현금으로 출금이 가능하고 주식 매수도 가능한 돈입니다.
3,"계좌 개설이 완료되었다면 그 이후 거래는 별다른 문제가 없을 것으로 보이며, 혹시라..."
4,삼성 증권은 비대면 계좌 개설 20일 제한이 없습니다.
...,...
1618,코넥스는 자본시장을 통한 초기 중소 벤처기업의 성장지원 및 모험자본 선순환 체계 구...
1619,비상장 주식 거래를 손쉽게 할 수 있도록 만들어진 곳이 라고 보면 될 것 같습니다....
1620,주식계좌에 주식구매대금을 이체하는 시점에 현금을 증여한 것으로 보아 이체일이 속하는...
1621,증여세 신고를 한 금전을 재원으로자녀 명의의 계좌에 공모주 투자시자녀 소유의 주식을...


## Vectorizing

In [ ]:
# TF-IDF 벡터화를 위한 객체를 생성합니다.
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
def get_top_similar_candidates(context, candidates, top_n=10):
    """
    주어진 'context'와 'candidates' 사이의 문장 유사도를 계산하고, 가장 유사한 후보들을 반환합니다.

    Args:
    - context (str): 주어진 문맥입니다.
    - candidates (dict): 후보 답변들을 담고 있는 사전 형태의 데이터 구조입니다.
        candidates['answer'] (list of str): 후보 답변들을 포함한 리스트입니다.
    - top_n (int, optional): 반환할 상위 유사한 후보의 수입니다. 기본값은 10입니다.

    Returns:
    - top_candidates (list of str): 가장 유사한 상위 후보 답변들의 리스트입니다.
    """

    # 후보 답변의 개수를 출력
    print("후보 답변 수:", len(candidates['answer']))

    # 유사도 값을 저장할 리스트 초기화
    lst = []

    # 각 후보 답변과 주어진 문맥 간의 유사도 계산
    for candidate in candidates['answer']:
        # 두 문장을 TF-IDF 벡터로 변환합니다.
        tfidf_matrix = tfidf_vectorizer.fit_transform([context, candidate])
        # 코사인 유사도를 계산합니다.
        cosine_sim = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])
        lst.append(cosine_sim)

    # 유사도 리스트 출력 및 리스트의 길이 출력
    print(lst)
    print("유사도 리스트 길이:", len(lst))

    # 유사도 값을 평탄화하고 정렬
    flattened_lst = [item.flatten()[0] for item in lst]
    sorted_indices = np.argsort(flattened_lst)[::-1][:top_n]

    # 상위 유사한 후보 출력
    for i in range(top_n):
        print("상위", top_n, "유사도:", lst[sorted_indices[i]])
        print("상위", top_n, "답변:", candidates['answer'][sorted_indices[i]])

    # 상위 유사한 후보 반환
    top_candidates = [candidates['answer'][index] for index in sorted_indices]

    return top_candidates

In [ ]:
def context_input(context):
    """
    입력된 문맥을 처리하여 토큰 ID와 입력 마스크를 생성합니다.

    Args:
        context (str): 처리할 문맥 문자열.

    Returns:
        contexts_token_ids_list_batch: 토큰 ID 텐서.
        contexts_input_masks_list_batch: 입력 마스크 텐서.
    """
    context_input_ids, context_input_masks = context_transform(context)
    contexts_token_ids_list_batch, contexts_input_masks_list_batch = [context_input_ids], [context_input_masks]

    long_tensors = [contexts_token_ids_list_batch, contexts_input_masks_list_batch]

    contexts_token_ids_list_batch, contexts_input_masks_list_batch = (torch.tensor(t, dtype=torch.long, device=device) for t in long_tensors)

    return contexts_token_ids_list_batch, contexts_input_masks_list_batch

def response_input(candidates):
    """
    후보 응답들을 처리하여 토큰 ID와 입력 마스크를 생성합니다.

    Args:
        candidates (list of str): 처리할 후보 응답 문자열들.

    Returns:
        responses_token_ids_list_batch: 토큰 ID 텐서.
        responses_input_masks_list_batch: 입력 마스크 텐서.
    """
    responses_token_ids_list, responses_input_masks_list = response_transform(candidates)
    responses_token_ids_list_batch, responses_input_masks_list_batch = [responses_token_ids_list], [responses_input_masks_list]

    long_tensors = [responses_token_ids_list_batch, responses_input_masks_list_batch]

    responses_token_ids_list_batch, responses_input_masks_list_batch = (torch.tensor(t, dtype=torch.long, device=device) for t in long_tensors)

    return responses_token_ids_list_batch, responses_input_masks_list_batch

def embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch):
    """
    문맥을 기반으로 임베딩을 생성합니다.

    Args:
        contexts_token_ids_list_batch (torch.Tensor): 문맥의 토큰 ID 텐서.
        contexts_input_masks_list_batch (torch.Tensor): 문맥의 입력 마스크 텐서.

    Returns:
         embs: 생성된 임베딩 텐서.
    """
    with torch.no_grad():
        model.eval()

        ctx_out = model.bert(contexts_token_ids_list_batch, contexts_input_masks_list_batch)[0]  # [bs, length, dim]
        poly_code_ids = torch.arange(model.poly_m, dtype=torch.long).to(contexts_token_ids_list_batch.device)
        poly_code_ids = poly_code_ids.unsqueeze(0).expand(1, model.poly_m)
        poly_codes = model.poly_code_embeddings(poly_code_ids) # [bs, poly_m, dim]
        embs = model.dot_attention(poly_codes, ctx_out, ctx_out) # [bs, poly_m, dim]

        return embs

def cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch):
    """
    응답 후보들의 임베딩을 생성합니다.

    Args:
        responses_token_ids_list_batch (torch.Tensor): 응답 후보들의 토큰 ID 텐서.
        responses_input_masks_list_batch (torch.Tensor): 응답 후보들의 입력 마스크 텐서.

    Returns:
        cand_emb: 생성된 임베딩 텐서.
    """
    with torch.no_grad():
        model.eval()

        batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape # res_cnt is 1 during training
        responses_token_ids_list_batch = responses_token_ids_list_batch.view(-1, seq_length)
        responses_input_masks_list_batch = responses_input_masks_list_batch.view(-1, seq_length)
        cand_emb = model.bert(responses_token_ids_list_batch, responses_input_masks_list_batch)[0][:,0,:] # [bs, dim]
        cand_emb = cand_emb.view(batch_size, res_cnt, -1) # [bs, res_cnt, dim]

        return cand_emb

def loss(embs, cand_emb, contexts_token_ids_list_batch, responses_token_ids_list_batch):
    """
    손실을 계산합니다.

    Args:
        embs (torch.Tensor): 문맥 임베딩 텐서.
        cand_emb (torch.Tensor): 응답 후보들의 임베딩 텐서.
        contexts_token_ids_list_batch (torch.Tensor): 문맥의 토큰 ID 텐서.
        responses_token_ids_list_batch (torch.Tensor): 응답 후보들의 토큰 ID 텐서.

    Returns:
        loss: 계산된 손실 값.
    """
    batch_size, res_cnt, seq_length = responses_token_ids_list_batch.shape

    ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, bs, dim]
    ctx_emb = ctx_emb.squeeze()
    dot_product = (ctx_emb*cand_emb) # [bs, bs]
    dot_product = dot_product.sum(-1)
    mask = torch.eye(batch_size).to(contexts_token_ids_list_batch.device) # [bs, bs]
    loss = F.log_softmax(dot_product, dim=-1)
    loss = loss * mask
    loss = (-loss.sum(dim=1))
    loss = loss.mean()

    return loss

def score(embs, cand_emb):
    """
    점수를 계산합니다.

    Args:
        embs (torch.Tensor): 문맥 임베딩 텐서.
        cand_emb (torch.Tensor): 응답 후보들의 임베딩 텐서.

    Returns:
        dot_product: 계산된 점수 텐서.
    """
    with torch.no_grad():
        model.eval()

        ctx_emb = model.dot_attention(cand_emb, embs, embs) # [bs, res_cnt, dim]
        dot_product = (ctx_emb*cand_emb).sum(-1)

        return dot_product


## Example

In [ ]:
context = input('질문을 입력하세요: ')
print('질문:', context)

top_similar_candidates = get_top_similar_candidates(context, data)
print('Top Similar Candidates:')


candidates = top_similar_candidates

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

PATH = '/content/drive/MyDrive/poly_16_pytorch_model.bin'

bert_name = 'klue/bert-base'
bert_config = BertConfig.from_pretrained(bert_name)

tokenizer = BertTokenizer.from_pretrained(bert_name)
tokenizer.add_tokens(['\n'], special_tokens=True)

context_transform = SelectionJoinTransform(tokenizer=tokenizer, max_len=256)
response_transform = SelectionSequentialTransform(tokenizer=tokenizer, max_len=128)

bert = BertModel.from_pretrained(bert_name, config=bert_config)

model = PolyEncoder(bert_config, bert=bert, poly_m=16)
model.resize_token_embeddings(len(tokenizer))
model.load_state_dict(torch.load(PATH))
model.to(device)
model.device

contexts_token_ids_list_batch, contexts_input_masks_list_batch = context_input(context)
responses_token_ids_list_batch, responses_input_masks_list_batch = response_input(candidates)
embs = embs_gen(contexts_token_ids_list_batch, contexts_input_masks_list_batch)
cand_emb = cand_emb_gen(responses_token_ids_list_batch, responses_input_masks_list_batch)
score_ = score(embs, cand_emb)
max_value, max_index = torch.max(score_, dim=1, keepdim=True)

print('답변:', candidates[max_index.item()])

Cloning into 'Poly-Encoder'...
remote: Enumerating objects: 129, done.
remote: Counting objects: 100% (121/121), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 129 (delta 67), reused 107 (delta 61), pack-reused 8
Receiving objects: 100% (129/129), 35.10 KiB | 5.01 MiB/s, done.
Resolving deltas: 100% (69/69), done.
/content/Poly-Encoder/Poly-Encoder/Poly-Encoder/Poly-Encoder/Poly-Encoder
질문을 입력하세요: 미성년자
질문: 미성년자
1623
[array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), array([[0.]]), ar

You are resizing the embedding layer without providing a `pad_to_multiple_of` parameter. This means that the new embedding dimension will be 32001. This might induce some performance reduction as *Tensor Cores* will not be available. For more details about this, or help on choosing the correct value for resizing, refer to this guide: https://docs.nvidia.com/deeplearning/performance/dl-performance-matrix-multiplication/index.html#requirements-tc


답변: 미성년자 계좌 계설은 부모님과 함께 가야합니다. 
